<a href="https://colab.research.google.com/github/JonathanAMichaels/Numerai_NN/blob/main/Numerai_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import copy
import torch.nn as nn
import pandas
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy import stats
from scipy.io import loadmat
from IPython.display import display, clear_output


# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU")
else:
    device = torch.device("cpu")
    print("CPU")

data = {}
f = h5py.File('TrainingData.mat')
for k, v in f.items():
    data[k] = np.array(v)

val_data = loadmat('ValData.mat')

#df=pandas.read_csv("/media/jonathan/Data/Numerai/numerai_training_data.csv")
#df.head()
# There are 501808 rows grouped into eras, and a single target (target_kazutsugi)
#df.shape

# There's 310 features
#features = [c for c in df if c.startswith("feature")]
#df["erano"] = df.era.str.slice(3).astype(int)
#eras = df.erano
#target = "target_kazutsugi"
#len(features)

#x = torch.tensor(df[features].values).type(torch.FloatTensor).to(device)
#y = torch.tensor(df[target].values).type(torch.FloatTensor).to(device)

x = torch.from_numpy(data.get('inpData')).transpose(0,1).type(torch.FloatTensor).to(device)
y = torch.from_numpy(data.get('targData')).transpose(0,1).type(torch.FloatTensor).to(device)

x_val1 = torch.from_numpy(val_data.get('inp_val1')).type(torch.FloatTensor).to(device)
x_val2 = torch.from_numpy(val_data.get('inp_val2')).type(torch.FloatTensor).to(device)
y_val1 = val_data.get('targ_val1')
y_val2 = val_data.get('targ_val2')
eras_val1 = val_data.get('eras_val1').transpose()
eras_val2 = val_data.get('eras_val2').transpose()
uEras1 = np.unique(eras_val1)
uEras2 = np.unique(eras_val2)

# The models should be scored based on the rank-correlation (spearman) with the target
#def numerai_score(y_true, y_pred):
#    rank_pred = y_pred.groupby(eras).apply(lambda x: x.rank(pct=True, method="first"))
#    return np.corrcoef(y_true, rank_pred)[0,1]

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 2000, x.shape[1], 4000, 1
# 500 batch size best, 4000 units best

GPU


/home/jonathan/.conda/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:22: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


In [ ]:
def myCrossEntropyLoss(outputs, labels):
    A = labels*torch.log(outputs+1e-200)
    B = (1-labels)*torch.log(1-outputs+1e-200)
    O = torch.mean(A+B)
    return -O

# Use the nn package to define our model and loss function.
H2 = 500
model = nn.Sequential(
    nn.Linear(D_in, H),
    nn.RReLU(),
    nn.Linear(H, D_out),
    nn.Sigmoid(),
)
model.to(device)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algorithms. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3 # 1e-4 best
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= 1e-5) # 1e-5 best

In [ ]:
fig = plt.figure(figsize=(9,3), dpi=100)
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

all_loss = []
all_sharpe = []
all_cor = []
all_ratio = []
all_t = []
best_sharpe = 0
for t in range(30000):
    l1_regularization, l2_regularization = torch.tensor(0).type(torch.FloatTensor).to(device), torch.tensor(0).type(torch.FloatTensor).to(device)
    batch_ind = np.random.randint(0,x.shape[0]-1,N)
    # Forward pass: compute predicted y by passing x to the model.
    model.train()
    y_pred = model(x[batch_ind,:])

    for param in model.parameters():
        l1_regularization += torch.norm(param, 1)**2
        l2_regularization += torch.norm(param, 2)**2

    # Compute loss.
    criterion = torch.nn.BCELoss()
    loss = criterion(y_pred, y[batch_ind])# + 1e-11*l1_regularization + 0*l2_regularization
    # 1e-11 best
    model.eval()
    [traincor, p] = stats.spearmanr(y_pred.cpu().detach().numpy(), y[batch_ind].cpu().detach().numpy())
    y_pred1 = model(x_val1)
    y_pred2 = model(x_val2)
    cor1 = []
    cor2 = []
    temp_pred = y_pred1.cpu().detach().numpy()
    for i in uEras1:
        [temp, p] = stats.spearmanr(y_val1[eras_val1 == i], temp_pred[eras_val1 == i])
        cor1.append(temp)
    temp_pred = y_pred2.cpu().detach().numpy()
    for i in uEras2:
        [temp, p] = stats.spearmanr(y_val2[eras_val2 == i], temp_pred[eras_val2 == i])
        cor2.append(temp)
    sharpe = np.mean(np.concatenate((cor1,cor2))) / np.std(np.concatenate((cor1, cor2)))
    if sharpe > best_sharpe:
        best_sharpe = sharpe
        best_model = copy.deepcopy(model)
        best_model.to(torch.device("cpu"))
    if (t % 20 == 0) and (t > 0):
        all_sharpe.append(sharpe)
        all_cor.append(np.mean(np.concatenate((cor1,cor2))))
        all_ratio.append((np.mean(cor1)-np.mean(cor2))/(np.mean(cor1)+np.mean(cor2)))
        all_t.append(t)
        ax1.cla()
        ax1.plot(all_t, all_sharpe)
        ax2.cla()
        ax2.plot(all_t, all_cor)
        ax3.cla()
        ax3.plot(all_t, all_ratio)
        display(fig)
        clear_output(wait = True)
        plt.pause(0.001)

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    model.train()
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

KeyboardInterrupt: 

In [ ]:
w0 = best_model.state_dict()['0.weight'].cpu().detach().numpy()
fig2 = plt.figure(figsize=(8,15), dpi=300)
ax2 = fig2.add_subplot(1,1,1)
ax2.imshow(w0)

In [ ]:
data = {}
f = h5py.File('TournamentData.mat')
for k, v in f.items():
    data[k] = np.array(v)
data = torch.from_numpy(data.get('saveInpData')).transpose(0,1).type(torch.FloatTensor)
y_pred = best_model(data)

/home/jonathan/.conda/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [ ]:
df = pandas.read_csv("/media/jonathan/Data/Numerai/numerai_tournament_data.csv")
df.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target_kazutsugi
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,...,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.00
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,...,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.25
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,...,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,...,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.75
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,...,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,1.00


In [ ]:
best_model.load_state_dict(torch.load('/media/jonathan/Data/Numerai/PYTORCH2/best_model'))
y_pred = best_model(data)

In [ ]:
df2 = pandas.DataFrame()
df2['id'] = df['id']
df2['prediction_kazutsugi'] = y_pred.detach().numpy()
df2.to_csv("/media/jonathan/Data/Numerai/PYTORCH2/pytorch_test.csv", index=False)
#torch.save(best_model.state_dict(), '/media/jonathan/Data/Numerai/PYTORCH1/best_model')